In [1]:
import pandas as pd
import time
import altair as alt
import paretoset

#alt.renderers.enable('png')
#alt.renderers.set_embed_options(scale_factor=3.0)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Reading Results

##### Before processing

In [2]:
price = pd.read_csv('Data/PriceCurve_SE4_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/pv_sam.csv')
load = pd.read_csv('Data/LoadCurve.csv', sep = ';')

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MW to kW
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor
data['solar_PV']= (data['solar_PV'])
data['Price']= (data['Price'] / 1000)
data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
#data.head(10)

In [3]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 200             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
bess_bos= 250               #https://www.energy-storage.news/li-ion-bess-costs-could-fall-47-by-2030-nrel-says-in-long-term-forecast-update/ in EUR/kW
## Also see: https://www.nrel.gov/grid/assets/pdfs/second_grid_sim_zagoras.pdf
# pv_opex= 17                 #EUR/kWh ->reference in excel
# bess_opex= 0.125            #EUR/kWh ->reference in excel
pv_opex= 3
bess_opex= 6

pv_co2=             33      #kgCO2eq/kW_powerDC ->reference in excel
bess_co2=           100     #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2=        0       #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2=      0       #kgCO2eq/kW_powerDC ->assumption
discount_rate=      0.0485  #assumption
degradation_rate=   0.025   #assumption (based on reaching 80% SoH in 8 years)
lifetime_project=   32      #for the project lifetime
lifetime_bess=      8       #for the BESS lifetime

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'bess_bos':         bess_bos,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate
}

In [4]:
def calc_lcoe(annual_output, capital_cost, annual_operating_cost, eol_burden, discount_rate, degradation_rate, lifetime):
    discount_factor_annualised = ( (1 - ( 1/( ( 1 + discount_rate ) **lifetime ) ) ) * ( 1 + discount_rate )) / ( ( 1 + discount_rate ) -1 )
    degradation_factor_annualised = ( (1 - ( 1/( ( 1 + degradation_rate ) **lifetime ) ) ) * ( 1 + degradation_rate )) / ( ( 1 + degradation_rate ) -1 )
    result = ( capital_cost + annual_operating_cost * discount_factor_annualised + eol_burden) / ( annual_output * degradation_factor_annualised )
    return result


def get_KPIs(
        profile,
        solar_factor,
        PessCH,
        EnESS
        # solar_factor=   pv_scale,
        # PessCH=         p_bess,
        # EnESS=          e_bess
    ):
    '''
    Base columns when profile is loaded:

    'Hour',
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
    'SoC',
    'sum_power_flows'

    '''
    price_curve= pd.read_csv(
        'Data/PriceCurve_SE3_2021.csv',
        delimiter = ';',
        usecols = ['Grid_Price']                        #EUR/MWh
    ) / 1000
    co2_curve= pd.read_csv(
        'Data/production_emissions.csv',
        delimiter = ',',
        usecols = ['carbon_intensity_production_avg']   #gCO2/kWh OR kgCO2/MWh
    ) / 1000
    # load= (
    #     pd.read_csv(
    #         'Data/LoadCurve.csv',
    #         sep= ';'
    #     )
    # ).sum()

    t_bess= EnESS/PessCH

    ## Add profits equation, other factors
    savings= 0
    for i in range(8760):
        savings += ((price_curve.Grid_Price[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
    
    energy_procurement_costs= 0
    for i in range(8760):
        energy_procurement_costs += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
    
    energy_procurement_costs_load= 0
    for i in range(8760):
        energy_procurement_costs_load += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_Load[i]))
    
    energy_procurement_costs_bess= 0
    for i in range(8760):
        energy_procurement_costs_bess += ((price_curve.Grid_Price[i]) * (profile.P_Grid_to_BESS[i]))

    profit= 0
    for i in range(8760):
        profit += ((price_curve.Grid_Price[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
    
    profit_pv= 0
    for i in range(8760):
        profit_pv += ((price_curve.Grid_Price[i]) * (profile.P_PV_to_Grid[i]))
    
    profit_bess= 0
    for i in range(8760):
        profit_bess += ((price_curve.Grid_Price[i]) * (profile.P_BESS_to_Grid[i]))
    
    co2avoided= 0
    for i in range(8760):
        #co2avoided += ((co2_curve.carbon_intensity_production_avg[i] - 12) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
        co2avoided += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i]))
    
    co2burden= 0
    for i in range(8760):
        co2burden += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i]))
    
    co2burden_load= 0
    for i in range(8760):
        co2burden_load += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_Load[i]))
    
    co2burden_bess= 0
    for i in range(8760):
        co2burden_bess += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_Grid_to_BESS[i]))

    co2abatement= 0
    for i in range(8760):
        co2abatement += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i]))
    
    co2abatement_pv= 0
    for i in range(8760):
        co2abatement_pv += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_PV_to_Grid[i]))
    
    co2abatement_bess= 0
    for i in range(8760):
        co2abatement_bess += ((co2_curve.carbon_intensity_production_avg[i]) * (profile.P_BESS_to_Grid[i]))
    
    

    pv2load=    profile['P_PV_to_Load'].sum()
    pv2bess=    profile['P_PV_to_BESS'].sum()
    pv2curtail= profile['P_PV_curtailment'].sum()
    pv2grid=    profile['P_PV_to_Grid'].sum()
    bess2load=  profile['P_BESS_to_Load'].sum()
    bess2grid=  profile['P_BESS_to_Grid'].sum()
    grid2load=  profile['P_Grid_to_Load'].sum()
    grid2bess=  profile['P_Grid_to_BESS'].sum()

    annual_energy= pv2load + bess2load + pv2grid + bess2grid
    #annual_energy=  data['solar_PV'].sum() * solar_factor
    annual_energy_enduser= pv2load + bess2load + grid2load

    if EnESS == 0:
        cycles= int(0)
    else:
        cycles= int((int(bess2load) + int(bess2grid)) / (EnESS * 1000))



    ## Re-check capex equations and make corrections (e.g. add BOS, BOP, EoL)
    ## If need be, reference specific equations for LCOE, or parts of LCOE, calculations
    capex= solar_factor * params['pv_price'] * 1000 + EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    capex_pv= solar_factor * params['pv_price'] * 1000
    capex_bess= EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    opex= solar_factor * params['pv_opex'] * 1000 + EnESS * 1000 * params['bess_opex']
    opex_pv= solar_factor * params['pv_opex'] * 1000
    opex_bess= EnESS * 1000 * params['bess_opex']
    opex_investor= opex + energy_procurement_costs_bess
    if EnESS == 0:
        opex_enduser= opex + energy_procurement_costs_load
    elif EnESS == 0.0:
        opex_enduser= opex + energy_procurement_costs_load
    else:
        opex_enduser= opex + energy_procurement_costs_load + energy_procurement_costs_bess * (bess2load/(bess2load + bess2grid))
    eol_capex= capex * 0.3
    capex_co2= solar_factor * params['pv_co2'] * 1000 + EnESS * 1000 * params['bess_co2']
    capex_co2_pv= solar_factor * params['pv_co2'] * 1000
    capex_co2_bess= EnESS * 1000 * params['bess_co2']
    opex_co2= solar_factor * params['pv_opex_co2'] * 1000 + EnESS * 1000 * params['bess_opex_co2']
    opex_co2_pv= solar_factor * params['pv_opex_co2'] * 1000
    opex_co2_bess= EnESS * 1000 * params['bess_opex_co2']
    opex_co2_investor= opex_co2 + co2burden_bess
    if EnESS == 0:
        opex_co2_enduser= opex_co2 + co2burden_load
    elif EnESS == 0.0:
        opex_co2_enduser= opex_co2 + co2burden_load
    else:
        opex_co2_enduser= opex_co2 + co2burden_load + co2burden_bess * (bess2load/(bess2load + bess2grid))
    eol_co2= capex_co2 * 0.3

    ## Get LCOE and LCO2 for the system
    if annual_energy == 0:
        lcoe_revenue=   0
        lcoe_investor=  0
        lco2_abatement= 0
        lco2_investor=  0
    elif annual_energy == 0.0:
        lcoe_revenue=   0
        lcoe_investor=  0
        lco2_abatement= 0
        lco2_investor=  0
    else:
        lcoe_revenue= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex,
            annual_operating_cost= (opex + energy_procurement_costs_bess - profit),
            eol_burden= eol_capex,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lcoe_investor= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex,
            annual_operating_cost= opex_investor,
            eol_burden= eol_capex,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lco2_abatement= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex_co2,
            annual_operating_cost= (opex_co2 + co2burden_bess - co2abatement),
            eol_burden= eol_co2,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )
        lco2_investor= calc_lcoe(
            annual_output= annual_energy,
            capital_cost= capex_co2,
            annual_operating_cost= opex_co2_investor,
            eol_burden= eol_co2,
            discount_rate= discount_rate,
            degradation_rate= degradation_rate,
            lifetime= 32
        )




    lcoe_enduser= calc_lcoe(
        annual_output= annual_energy_enduser,
        capital_cost= capex,
        #annual_operating_cost= (opex + energy_procurement_costs_load + energy_procurement_costs_bess * (bess2load/(bess2load + bess2grid)) - profit - savings),
        annual_operating_cost= opex_enduser,
        eol_burden= eol_capex,
        discount_rate= discount_rate,
        degradation_rate= degradation_rate,
        lifetime= 32
    )
    
    lco2_enduser= calc_lcoe(
        annual_output= annual_energy_enduser,
        capital_cost= capex_co2,
        #annual_operating_cost= (opex_co2 + co2burden_load + co2burden_bess * (bess2load/(bess2load + bess2grid)) - co2abatement - co2avoided),
        annual_operating_cost= opex_co2_enduser,
        eol_burden= eol_co2,
        discount_rate= discount_rate,
        degradation_rate= degradation_rate,
        lifetime= 32
    )



    # lcoe_full = calc_lcoe(
    #     annual_output= annual_energy,
    #     capital_cost= capex,
    #     annual_operating_cost= (opex + energy_procurement_costs),
    #     #annual_operating_cost= opex,
    #     eol_burden= eol_capex,
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lcoe_btm = calc_lcoe(
    #     annual_output= (pv2load + bess2load),
    #     capital_cost= (capex * (grid2load / (grid2load + grid2bess))),
    #     annual_operating_cost= ((opex) * (grid2load / (grid2load + grid2bess)) + energy_procurement_costs_load),
    #     eol_burden= ((capex * (grid2load / (grid2load + grid2bess))) * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lcoe_pv = calc_lcoe(
    #     annual_output= (pv2load + pv2grid + pv2bess),
    #     capital_cost= capex_pv,
    #     annual_operating_cost= opex_pv,
    #     eol_burden= (capex_pv * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lcos = calc_lcoe(
    #     annual_output= (bess2grid + bess2load),
    #     capital_cost= capex_bess,
    #     annual_operating_cost= (opex_bess + energy_procurement_costs_bess),
    #     eol_burden= (capex_bess * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lcoe_arbitrage = calc_lcoe(
    #     annual_output= (pv2grid + bess2grid),
    #     capital_cost= (capex * (grid2bess/(grid2bess + grid2load))),
    #     annual_operating_cost= (opex * (grid2bess / (grid2load + grid2bess)) + energy_procurement_costs_bess),
    #     eol_burden= (capex * (grid2bess/(grid2bess + grid2load)) * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )

    
    # lco2_full = calc_lcoe(
    #     annual_output= annual_energy,
    #     capital_cost= capex,
    #     annual_operating_cost= (opex + co2burden),
    #     #annual_operating_cost= opex,
    #     eol_burden= eol_capex,
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lco2_btm = calc_lcoe(
    #     annual_output= (pv2load + bess2load),
    #     capital_cost= (capex_co2 * (grid2load / (grid2load + grid2bess))),
    #     annual_operating_cost= ((opex_co2) * (grid2load / (grid2load + grid2bess)) + co2burden_load),
    #     eol_burden= ((capex_co2 * (grid2load / (grid2load + grid2bess))) * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lco2_pv = calc_lcoe(
    #     annual_output= (pv2load + pv2grid + pv2bess),
    #     capital_cost= capex_co2_pv,
    #     annual_operating_cost= opex_co2_pv,
    #     eol_burden= (capex_co2_pv * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lco2_bess = calc_lcoe(
    #     annual_output= (bess2grid + bess2load),
    #     capital_cost= capex_co2_bess,
    #     annual_operating_cost= (opex_co2_bess + co2burden_bess),
    #     eol_burden= (capex_co2_bess * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    # lco2_arbitrage = calc_lcoe(
    #     annual_output= (pv2grid + bess2grid),
    #     capital_cost= (capex_co2 * (grid2bess/(grid2bess + grid2load))),
    #     annual_operating_cost= (opex_co2 * (grid2bess / (grid2load + grid2bess)) + co2burden_bess),
    #     eol_burden= (capex_co2 * (grid2bess/(grid2bess + grid2load)) * 0.3),
    #     discount_rate= discount_rate,
    #     degradation_rate= degradation_rate,
    #     lifetime= 32
    # )
    
    


    #payback_period = (dataframe.P_PV * params['pv_price'] * 1000 + dataframe.E_BESS * 1000 * params['bess_price']) / (dataframe.savings_EUR - (dataframe.P_PV * params['pv_opex'] * 1000 + dataframe.E_BESS * 1000 * params['bess_opex']))
    
    
    ## SS = how much of the load was covered by the PV system
    self_sufficiency= ((pv2load + bess2load) / data.Load.sum()) * 100
    #self_sufficiency= ((pv2load + pv2bess) / data.Load.sum()) * 100
    if EnESS == 0:
        ress= (pv2load / data.Load.sum()) * 100
    elif EnESS == 0.0:
        ress= (pv2load / data.Load.sum()) * 100
    elif solar_factor == 0:
        ress = 0
    elif solar_factor == 0.0:
        ress = 0
    else:
        ress= ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100

    ## SC = 1 - curtailment factor?
    if solar_factor == 0:
        self_consumption = 0
    else:
        self_consumption= ((pv2load + pv2bess) / (data.solar_PV.sum() * (solar_factor * 1000))) * 100
    
    # dataframe['curtailment_factor'] = ((dataframe.pv_curtailed) / (data.solar_PV.sum() * (dataframe.P_PV / 100))) * 100

    # dataframe['offset_period'] = (dataframe.P_PV * params['pv_co2'] * 1000 + dataframe.E_BESS * 1000 * params['bess_co2']) / dataframe.co2avoided_kgCO2
    
    # dataframe['capacity_factor'] = ((dataframe.pv2load + dataframe.bess2load)/(dataframe.P_PV * 8760)) * 100

    # dataframe['pv_load_ratio'] = dataframe.P_PV / 3.0

    # dataframe['bess_load_ratio'] = dataframe.bess_duration / 3.0

    # dataframe['bess_pv_ratio'] = dataframe.bess_duration / dataframe.P_PV
    
    
    parameters = (
        solar_factor,
        PessCH,
        EnESS,
        t_bess,
        pv2load,
        pv2bess,
        pv2curtail,
        pv2grid,
        bess2load,
        bess2grid,
        grid2load,
        grid2bess,
        annual_energy,
        capex,
        capex_pv,
        capex_bess,
        opex,
        opex_pv,
        opex_bess,
        capex_co2,
        capex_co2_pv,
        capex_co2_bess,
        opex_co2,
        opex_co2_pv,
        opex_co2_bess,
        #lcoe_full,
        #lcoe_btm,
        #lcoe_pv,
        #lcos,
        #lcoe_arbitrage,
        lcoe_investor,
        lcoe_enduser,
        lcoe_revenue,
        #lco2_full,
        #lco2_btm,
        #lco2_pv,
        #lco2_bess,
        #lco2_arbitrage,
        lco2_investor,
        lco2_enduser,
        lco2_abatement,
        savings,
        energy_procurement_costs,
        energy_procurement_costs_load,
        energy_procurement_costs_bess,
        profit,
        profit_pv,
        profit_bess,
        co2avoided,
        co2burden,
        co2burden_load,
        co2burden_bess,
        co2abatement,
        co2abatement_pv,
        co2abatement_bess,
        cycles,
        self_sufficiency,
        ress,
        self_consumption
    )
    return parameters

In [5]:
results_econ=   pd.DataFrame(
    columns=[
        'P_PV',
        'P_BESS',
        'E_BESS',
        'T_BESS',
        'pv2load',
        'pv2bess',
        'pv2curtail',
        'pv2grid',
        'bess2load',
        'bess2grid',
        'grid2load',
        'grid2bess',
        'annual_energy',
        'capex',
        'capex_pv',
        'capex_bess',
        'opex',
        'opex_pv',
        'opex_bess',
        'capex_co2',
        'capex_co2_pv',
        'capex_co2_bess',
        'opex_co2',
        'opex_co2_pv',
        'opex_co2_bess',
        #'lcoe_full',
        #'lcoe_btm',
        #'lcoe_pv',
        #'lcos',
        #'lcoe_arbitrage',
        'lcoe_investor',
        'lcoe_enduser',
        'lcoe_revenue',
        #'lco2_full',
        #'lco2_btm',
        #'lco2_pv',
        #'lco2_bess',
        #'lco2_arbitrage',
        'lco2_investor',
        'lco2_enduser',
        'lco2_abatement',
        'savings',
        'energy_procurement_costs',
        'energy_procurement_costs_load',
        'energy_procurement_costs_bess',
        'profit',
        'profit_pv',
        'profit_bess',
        'co2avoided',
        'co2burden',
        'co2burden_load',
        'co2burden_bess',
        'co2abatement',
        'co2abatement_pv',
        'co2abatement_bess',
        'cycles',
        'self_sufficiency',
        'ress',
        'self_consumption'
    ]
)
results_env=    pd.DataFrame(
    columns=[
        'P_PV',
        'P_BESS',
        'E_BESS',
        'T_BESS',
        'pv2load',
        'pv2bess',
        'pv2curtail',
        'pv2grid',
        'bess2load',
        'bess2grid',
        'grid2load',
        'grid2bess',
        'annual_energy',
        'capex',
        'capex_pv',
        'capex_bess',
        'opex',
        'opex_pv',
        'opex_bess',
        'capex_co2',
        'capex_co2_pv',
        'capex_co2_bess',
        'opex_co2',
        'opex_co2_pv',
        'opex_co2_bess',
        #'lcoe_full',
        #'lcoe_btm',
        #'lcoe_pv',
        #'lcos',
        #'lcoe_arbitrage',
        'lcoe_investor',
        'lcoe_enduser',
        'lcoe_revenue',
        #'lco2_full',
        #'lco2_btm',
        #'lco2_pv',
        #'lco2_bess',
        #'lco2_arbitrage',
        'lco2_investor',
        'lco2_enduser',
        'lco2_abatement',
        'savings',
        'energy_procurement_costs',
        'energy_procurement_costs_load',
        'energy_procurement_costs_bess',
        'profit',
        'profit_pv',
        'profit_bess',
        'co2avoided',
        'co2burden',
        'co2burden_load',
        'co2burden_bess',
        'co2abatement',
        'co2abatement_pv',
        'co2abatement_bess',
        'cycles',
        'self_sufficiency',
        'ress',
        'self_consumption'
    ]
)




##### Processing

In [6]:
# read_pv=        [0, 1, 3, 10]
# read_bess=      [1, 3, 10]
# read_time=      [0, 2, 4]
optimise_for=   ['price', 'co2']
pv_multiples= [
    7
]
bess_multiples= [
    2
]
bess_duration= [
    1
]

# ## Reading Scenarios and postprocessing into DataPoints
# start_time= time.time()

# # for pv_scale in read_pv:
# #     if pv_scale == 0:
# #         for p_bess in read_bess:
# #             for t_bess in [2, 4]:
# #                 for curve in optimise_for:
# #                     e_bess= t_bess * p_bess

# #                     profile = (pd.read_csv(f'Results/Batch4/CaseB/{curve}_{pv_scale}_{p_bess}_{t_bess}.csv')).fillna(value= 0)
# #                     profile['P_PV_to_Load']=    profile['P_PV_to_Load'].astype('int')
# #                     profile['P_PV_to_BESS']=    profile['P_PV_to_BESS'].astype('int')
# #                     profile['P_PV_curtailment']=profile['P_PV_curtailment'].astype('int')
# #                     profile['P_PV_to_Grid']=    profile['P_PV_to_Grid'].astype('int')
# #                     profile['P_BESS_to_Load']=  profile['P_BESS_to_Load'].astype('int')
# #                     profile['P_BESS_to_Grid']=  profile['P_BESS_to_Grid'].astype('int')
# #                     profile['P_Grid_to_Load']=  profile['P_Grid_to_Load'].astype('int')
# #                     profile['P_Grid_to_BESS']=  profile['P_Grid_to_BESS'].astype('int')
# #                     #print(profile.info())

# #                     if curve == 'price':
# #                         results_econ.loc[len(results_econ)] = (get_KPIs(profile, solar_factor= pv_scale, PessCH= p_bess, EnESS= e_bess))
# #                     elif curve == 'co2':
# #                         results_env.loc[len(results_env)] = (get_KPIs(profile, solar_factor= pv_scale, PessCH= p_bess, EnESS= e_bess))
# #                     else:
# #                         print('Check file name')
# #     elif pv_scale != 0:
# #         for p_bess in read_bess:
# #             for t_bess in read_time:
# #                 for curve in optimise_for:
# #                     e_bess= t_bess * p_bess

# #                     profile = (pd.read_csv(f'Results/Batch4/CaseB/{curve}_{pv_scale}_{p_bess}_{t_bess}.csv')).fillna(value= 0)
# #                     profile['P_PV_to_Load']=    profile['P_PV_to_Load'].astype('int')
# #                     profile['P_PV_to_BESS']=    profile['P_PV_to_BESS'].astype('int')
# #                     profile['P_PV_curtailment']=profile['P_PV_curtailment'].astype('int')
# #                     profile['P_PV_to_Grid']=    profile['P_PV_to_Grid'].astype('int')
# #                     profile['P_BESS_to_Load']=  profile['P_BESS_to_Load'].astype('int')
# #                     profile['P_BESS_to_Grid']=  profile['P_BESS_to_Grid'].astype('int')
# #                     profile['P_Grid_to_Load']=  profile['P_Grid_to_Load'].astype('int')
# #                     profile['P_Grid_to_BESS']=  profile['P_Grid_to_BESS'].astype('int')
# #                     #print(profile.info())

# #                     if curve == 'price':
# #                         results_econ.loc[len(results_econ)] = (get_KPIs(profile, solar_factor= pv_scale, PessCH= p_bess, EnESS= e_bess))
# #                     elif curve == 'co2':
# #                         results_env.loc[len(results_env)] = (get_KPIs(profile, solar_factor= pv_scale, PessCH= p_bess, EnESS= e_bess))
# #                     else:
# #                         print('Check file name')





# for curve in optimise_for:
#     for pv_multiple in pv_multiples:
#         for bess_multiple in bess_multiples:
#             for bess_cap in bess_duration:
#                 try:
#                     solar_factor= pv_multiple * 3
#                     PessCH= bess_multiple * 3
#                     EnESS= PessCH * bess_cap


#                     profile = (pd.read_csv(f'Results/Batch5/CaseB/{curve}_{pv_multiple}_{bess_multiple}_{bess_cap}.csv')).fillna(value= 0)
#                     profile['P_PV_to_Load']=        profile['P_PV_to_Load'].astype('int')
#                     profile['P_PV_to_BESS']=        profile['P_PV_to_BESS'].astype('int')
#                     profile['P_PV_curtailment']=    profile['P_PV_curtailment'].astype('int')
#                     profile['P_PV_to_Grid']=        profile['P_PV_to_Grid'].astype('int')
#                     profile['P_BESS_to_Load']=      profile['P_BESS_to_Load'].astype('int')
#                     profile['P_BESS_to_Grid']=      profile['P_BESS_to_Grid'].astype('int')
#                     profile['P_Grid_to_Load']=      profile['P_Grid_to_Load'].astype('int')
#                     profile['P_Grid_to_BESS']=      profile['P_Grid_to_BESS'].astype('int')
#                     #print(profile.info())

#                     if curve == 'price':
#                         results_econ.loc[len(results_econ)] = (get_KPIs(profile, solar_factor= solar_factor, PessCH= PessCH, EnESS= EnESS))
#                     elif curve == 'co2':
#                         results_env.loc[len(results_env)] = (get_KPIs(profile, solar_factor= solar_factor, PessCH= PessCH, EnESS= EnESS))
#                 except FileNotFoundError:
#                     continue




# end_time= time.time()
# execution_time= end_time - start_time

# # Convert execution time to hours, minutes, and seconds
# hours, remainder = divmod(execution_time, 3600)
# minutes, seconds = divmod(remainder, 60)

# to_read=(len(pv_multiples) * len(bess_multiples) * len(bess_duration) * len(optimise_for))
# been_read=   (len(results_econ) + len(results_env))
# # Print the execution time in the format hh:mm:ss
# print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
# print(f'Cases to read: {to_read}')
# print(f'Cases Read: {been_read}')
# print(f'Scenarios Generated: {100 * round((been_read/to_read), 4)}%')

In [7]:
# try:
#     results_econ= pd.read_csv(
#         'Results/Batch5/CaseB/results_econ.csv',
#         sep= ';',
#     ).drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
#     results_env= pd.read_csv(
#         'Results/Batch5/CaseB/results_env.csv',
#         sep= ';',
#     ).drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
#     results= pd.read_csv(
#         'Results/Batch5/CaseB/results.csv',
#         sep= ';',
#     ).drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
# except KeyError:
#     results_econ= pd.read_csv(
#         'Results/Batch5/CaseB/results_econ.csv',
#         sep= ';',
#     )
#     results_env= pd.read_csv(
#         'Results/Batch5/CaseB/results_env.csv',
#         sep= ';',
#     )
#     results= pd.read_csv(
#         'Results/Batch5/CaseB/results.csv',
#         sep= ';',
#     )





results_econ= pd.read_csv(
    'Results/Batch5/CaseB/results_econ.csv',
    sep= ';',
)
results_env= pd.read_csv(
    'Results/Batch5/CaseB/results_env.csv',
    sep= ';',
)
results= pd.read_csv(
    'Results/Batch5/CaseB/results.csv',
    sep= ';',
)

##### Cleaning Processed Results

In [8]:
def replace_value_in_column(df, column_name, old_value, new_value):
    df[column_name] = df[column_name].replace(old_value, new_value)
    return df

def remove_rows_with_specific_value(df, column_name, value):
    return df[df[column_name] != value]

def remove_rows_above_specific_value(df, column_name, value):
    return df[df[column_name] <= value]

def remove_rows_below_specific_value(df, column_name, value):
    return df[df[column_name] >= value]

def get_pareto_front(point, points):
    return np.any(np.all(points <= point, axis=1)) and np.any(np.all(points > point, axis=1))

In [9]:
results_econ['net_savings'] = results_econ['savings'] - results_econ['energy_procurement_costs']
results_env['net_savings'] = results_env['savings'] - results_econ['energy_procurement_costs']

results_econ['net_co2avoided'] = results_econ['co2avoided'] - results_econ['co2burden']
results_env['net_co2avoided'] = results_env['co2avoided'] - results_econ['co2burden']

#results_econ

In [10]:
results_econ['net_earnings'] = results_econ['profit'] + results_econ['savings'] - results_econ['energy_procurement_costs']
results_env['net_earnings'] = results_econ['profit'] + results_env['savings'] - results_econ['energy_procurement_costs']

results_econ['net_abatement'] = results_econ['co2abatement'] + results_econ['co2avoided'] - results_econ['co2burden']
results_env['net_abatement'] = results_econ['co2abatement'] + results_env['co2avoided'] - results_econ['co2burden']

#results_econ

In [11]:
def get_multiples(results):
    results['PV_Load_M']= results['P_PV']/3
    results['BESS_Load_M']= results['P_BESS']/3
    return

def clean_noStorage(results):
    for i in range(len(results)):
        if results['T_BESS'][i] == 0:
            results['P_BESS'][i] = 0
        else:
            continue
    return

def pv_share(results):
    results['pv2load_share']= results['pv2load']/(results['pv2load'] + results['pv2bess'] + results['pv2grid'] + results['pv2curtail'])
    results['pv2bess_share']= results['pv2bess']/(results['pv2load'] + results['pv2bess'] + results['pv2grid'] + results['pv2curtail'])
    results['pv2grid_share']= results['pv2grid']/(results['pv2load'] + results['pv2bess'] + results['pv2grid'] + results['pv2curtail'])
    results['curtailment_share']= results['pv2curtail']/(results['pv2load'] + results['pv2bess'] + results['pv2grid'] + results['pv2curtail'])
    return

def bess_share(results):
    results['bess2load_share']= results['bess2load']/(results['bess2grid'] + results['bess2load'])
    results['bess2grid_share']= results['bess2grid']/(results['bess2grid'] + results['bess2load'])
    return

In [12]:
# get_multiples(results_econ)
# get_multiples(results_env)

# clean_noStorage(results_econ)
# clean_noStorage(results_env)

# pv_share(results_econ)
# pv_share(results_env)

# bess_share(results_econ)
# bess_share(results_env)


# results_econ= results_econ.fillna(value= 0)
# results_env= results_env.fillna(value= 0)

# results_econ['PV_Load_M']= results_econ['PV_Load_M'].round(1)
# results_env['PV_Load_M']= results_env['PV_Load_M'].round(1)
# results_econ['BESS_Load_M']= results_econ['BESS_Load_M'].round(1)
# results_env['BESS_Load_M']= results_env['BESS_Load_M'].round(1)
# #results_econ

In [13]:
# results_econ['pv2load']=    (results_econ['pv2load']).astype('int')
# results_econ['pv2bess']=    (results_econ['pv2bess']).astype('int')
# results_econ['pv2curtail']= (results_econ['pv2curtail']).astype('int')
# results_econ['pv2grid']=    (results_econ['pv2grid']).astype('int')
# results_econ['bess2load']=  (results_econ['bess2load']).astype('int')
# results_econ['bess2grid']=  (results_econ['bess2grid']).astype('int')
# results_econ['grid2load']=  (results_econ['grid2load']).astype('int')
# results_econ['grid2bess']=  (results_econ['grid2bess']).astype('int')

# results_env['pv2load']=    (results_env['pv2load']).astype('int')
# results_env['pv2bess']=    (results_env['pv2bess']).astype('int')
# results_env['pv2curtail']= (results_env['pv2curtail']).astype('int')
# results_env['pv2grid']=    (results_env['pv2grid']).astype('int')
# results_env['bess2load']=  (results_env['bess2load']).astype('int')
# results_env['bess2grid']=  (results_env['bess2grid']).astype('int')
# results_env['grid2load']=  (results_env['grid2load']).astype('int')
# results_env['grid2bess']=  (results_env['grid2bess']).astype('int')

# #results_econ

In [14]:
# df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
# print(df)
# # Potentially problematic (could raise the warning)
# #df['A'][df['A'] > 1] = 10

# # Better approach
# df.loc[df['A'] > 1, 'C'] = 10
# print(df)

In [15]:
results_econ= results_econ.drop_duplicates(
    subset= [
        'P_PV',
        'P_BESS',
        'E_BESS',
        'T_BESS'
    ]
)
results_env= results_env.drop_duplicates(
    subset= [
        'P_PV',
        'P_BESS',
        'E_BESS',
        'T_BESS'
    ]
)

In [16]:
def classify_earnings(value):
    if value > 0:
        return 'positive'
    elif value < 0:
        return 'negative'
    else:
        return 'zero'

def classify_abatement(value):
    if value > 0:
        return 'positive'
    elif value < 0:
        return 'negative'
    else:
        return 'zero'

# Add a new column 'earnings_class' based on 'net_earnings'
results_econ['earnings_class'] = results_econ['net_earnings'].apply(classify_earnings)
results_env['earnings_class'] = results_env['net_earnings'].apply(classify_earnings)

results_econ['co2abatement_class'] = results_econ['net_abatement'].apply(classify_abatement)
results_env['co2abatement_class'] = results_env['net_abatement'].apply(classify_abatement)

results_econ['savings_class'] = results_econ['net_savings'].apply(classify_earnings)
results_env['savings_class'] = results_env['net_savings'].apply(classify_earnings)

results_econ['co2avoided_class'] = results_econ['net_co2avoided'].apply(classify_abatement)
results_env['co2avoided_class'] = results_env['net_co2avoided'].apply(classify_abatement)

In [17]:
#results_econ['earnings_class']
#results_econ['co2abatement_class']

In [18]:
# results_econ['label']= 'Economic Dispatch Optimisation'
# results_env['label']= 'Environmental Dispatch Optimisation'

# results= pd.merge(results_econ, results_env, how= 'outer')
# #results

##### Saving Processed Results

In [19]:
# results_econ.to_csv(
#     'Results/Batch5/CaseB/results_econ.csv',
#     sep= ';',
#     index= False
# )
# results_env.to_csv(
#     'Results/Batch5/CaseB/results_env.csv',
#     sep= ';',
#     index= False
# )
# results.to_csv(
#     'Results/Batch5/CaseB/results.csv',
#     sep= ';',
#     index= False
# )

# Plots - CaseB RES-Coupling + Energy Arbitrage

In [20]:
import pandas as pd
import altair as alt
import paretoset

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [21]:
# Leer resultados
results_econ = pd.read_csv('Results/Batch5/CaseB_autor_lowram/results_econ.csv', sep=';')
results_env = pd.read_csv('Results/Batch5/CaseB_autor_lowram/results_env.csv', sep=';')
results = pd.read_csv('Results/Batch5/CaseB_autor_lowram/results.csv', sep=';')

In [22]:
# Price and CO2eq bands
band_data_quartiles = pd.DataFrame({
    'Avg_Price': [0.033687, 0.079043],
    'Avg_CO2eq': [0.034770, 0.040760]
})
band_data_avg = pd.DataFrame({
    'Avg_Price': [0.065847],
    'Avg_CO2eq': [0.037910]
})

# Plot with error band
chart_band_price = alt.Chart(band_data_quartiles).mark_errorband(
    interpolate= 'linear',
    extent= 'ci',
    opacity= 0.25
).encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_price_quartiles = alt.Chart(band_data_quartiles).mark_rule(
    opacity= 0.75,
    strokeDash= (5, 5)
).encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_price_avg = alt.Chart(band_data_avg).mark_rule().encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_band_co2 = alt.Chart(band_data_quartiles).mark_errorband(
    interpolate= 'linear',
    extent= 'ci',
    opacity= 0.25
).encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_co2_quartiles = alt.Chart(band_data_quartiles).mark_rule(
    opacity= 0.75,
    strokeDash= (5, 5)
).encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_co2_avg = alt.Chart(band_data_avg).mark_rule().encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)

#chart_band_price + chart_rule_price

### Raw Plots

In [23]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
# color = alt.condition(
#     alt.datum.earnings_class == 'positive',  # Condition 1
#     alt.value(color_mapping_earnings['positive']),  # Color for positive
#     alt.condition(
#         alt.datum.earnings_class == 'negative', 
#         alt.value(color_mapping_earnings['negative']),  # Color for negative
#         alt.value(color_mapping_earnings['zero'])      # Default color (zero)
#     )
# )





lines_lcoe_revenue_econ= alt.Chart(results_econ, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_revenue_env= alt.Chart(results_env, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_revenue= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_abatement_econ= alt.Chart(results_econ, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_abatement_env= alt.Chart(results_env, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_abatement= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.15
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe_revenue)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [24]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}


lines_lcoe_investor_econ= alt.Chart(results_econ, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_investor_env= alt.Chart(results_env, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_investor_econ= alt.Chart(results_econ, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_investor_env= alt.Chart(results_env, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [25]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}

lines_lcoe_enduser_econ= alt.Chart(results_econ, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_enduser_env= alt.Chart(results_env, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'savings:Q',
        legend= alt.Legend(
            title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='blues',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_enduser_econ= alt.Chart(results_econ, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_enduser_env= alt.Chart(results_env, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'co2avoided:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='yellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [26]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)),#.stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [27]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [28]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [29]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_pv', 'capex_bess', 'opex_pv', 'opex_bess', 'energy_procurement_costs_load', 'energy_procurement_costs_bess']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity Expenditures - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Expenditures')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [30]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_pv', 'capex_bess', 'opex_pv', 'opex_bess', 'energy_procurement_costs_load', 'energy_procurement_costs_bess']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity Expenditures - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Expenditures')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_co2_pv', 'capex_co2_bess', 'opex_co2_pv', 'opex_co2_bess', 'co2burden_load', 'co2burden_bess']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity CO2eq Impact Sources - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'CO2eq Impact Sources')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [32]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_co2_pv', 'capex_co2_bess', 'opex_co2_pv', 'opex_co2_bess', 'co2burden_load', 'co2burden_bess']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity CO2eq Impact Sources - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Co2eq Impact Sources')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 30,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Quick-Extraction Pareto Front

In [33]:
# Assuming you want to minimize both objectives
pareto_points_lcoe_revenue_1 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_revenue_2 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_abatement_1 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_abatement_2 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_investor_1 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_investor_2 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_investor_1 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_investor_2 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_enduser_1 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_enduser_2 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_enduser_1 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_enduser_2 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_revenue = pd.merge(results.iloc[pareto_points_lcoe_revenue_1], results.iloc[pareto_points_lcoe_revenue_2], how='outer')
pareto_front_lco2_abatement = pd.merge(results.iloc[pareto_points_lco2_abatement_1], results.iloc[pareto_points_lco2_abatement_2], how='outer')

pareto_front_lcoe_investor = pd.merge(results.iloc[pareto_points_lcoe_investor_1], results.iloc[pareto_points_lcoe_investor_2], how='outer')
pareto_front_lco2_investor = pd.merge(results.iloc[pareto_points_lco2_investor_1], results.iloc[pareto_points_lco2_investor_2], how='outer')

pareto_front_lcoe_enduser = pd.merge(results.iloc[pareto_points_lcoe_enduser_1], results.iloc[pareto_points_lcoe_enduser_2], how='outer')
pareto_front_lco2_enduser = pd.merge(results.iloc[pareto_points_lco2_enduser_1], results.iloc[pareto_points_lco2_enduser_2], how='outer')

In [34]:
pareto_lcoe_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_revenue:Q')
)
pareto_lco2_abatement= alt.Chart(pareto_front_lco2_abatement).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_abatement:Q')
)

pareto_lcoe_investor= alt.Chart(pareto_front_lcoe_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_investor:Q')
)
pareto_lco2_investor= alt.Chart(pareto_front_lco2_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_investor:Q')
)

pareto_lcoe_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_enduser:Q')
)
pareto_lco2_enduser= alt.Chart(pareto_front_lco2_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_enduser:Q')
)

In [35]:
alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env,
        pareto_lcoe_revenue
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env,
        pareto_lco2_abatement
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [36]:
alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env,
        pareto_lcoe_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env,
        pareto_lco2_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [37]:
alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg,
        pareto_lcoe_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg,
        pareto_lco2_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [38]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_revenue,
        right_up_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_revenue,
        right_down_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [39]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [40]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

### Cut-off 120 EUR/MWh
(Since the largest point, that is still on the Paretto front, has an LCOE between 110 and 120 EUR/MWh)

In [41]:
results_econ_copy= results_econ.copy(deep= True)
results_env_copy= results_env.copy(deep= True)
results_copy= results.copy(deep= True)

results_econ= remove_rows_above_specific_value(results_econ, 'lcoe_investor', 0.12)
results_env= remove_rows_above_specific_value(results_env, 'lcoe_investor', 0.12)
results= remove_rows_above_specific_value(results, 'lcoe_investor', 0.12)

In [42]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
# color = alt.condition(
#     alt.datum.earnings_class == 'positive',  # Condition 1
#     alt.value(color_mapping_earnings['positive']),  # Color for positive
#     alt.condition(
#         alt.datum.earnings_class == 'negative', 
#         alt.value(color_mapping_earnings['negative']),  # Color for negative
#         alt.value(color_mapping_earnings['zero'])      # Default color (zero)
#     )
# )





lines_lcoe_revenue_econ= alt.Chart(results_econ, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_revenue_env= alt.Chart(results_env, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_revenue= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_abatement_econ= alt.Chart(results_econ, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_abatement_env= alt.Chart(results_env, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_abatement= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.15
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe_revenue)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [43]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}


lines_lcoe_investor_econ= alt.Chart(results_econ, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_investor_env= alt.Chart(results_env, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_investor_econ= alt.Chart(results_econ, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_investor_env= alt.Chart(results_env, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [44]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}

lines_lcoe_enduser_econ= alt.Chart(results_econ, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_enduser_env= alt.Chart(results_env, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'savings:Q',
        legend= alt.Legend(
            title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='blues',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_enduser_econ= alt.Chart(results_econ, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_enduser_env= alt.Chart(results_env, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'co2avoided:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='yellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [45]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [46]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Quick-Extraction Pareto Front

In [47]:
# Assuming you want to minimize both objectives
pareto_points_lcoe_revenue_1 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_revenue_2 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_abatement_1 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_abatement_2 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_investor_1 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_investor_2 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_investor_1 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_investor_2 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_enduser_1 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_enduser_2 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_enduser_1 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_enduser_2 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_revenue = pd.merge(results.iloc[pareto_points_lcoe_revenue_1], results.iloc[pareto_points_lcoe_revenue_2], how='outer')
pareto_front_lco2_abatement = pd.merge(results.iloc[pareto_points_lco2_abatement_1], results.iloc[pareto_points_lco2_abatement_2], how='outer')

pareto_front_lcoe_investor = pd.merge(results.iloc[pareto_points_lcoe_investor_1], results.iloc[pareto_points_lcoe_investor_2], how='outer')
pareto_front_lco2_investor = pd.merge(results.iloc[pareto_points_lco2_investor_1], results.iloc[pareto_points_lco2_investor_2], how='outer')

pareto_front_lcoe_enduser = pd.merge(results.iloc[pareto_points_lcoe_enduser_1], results.iloc[pareto_points_lcoe_enduser_2], how='outer')
pareto_front_lco2_enduser = pd.merge(results.iloc[pareto_points_lco2_enduser_1], results.iloc[pareto_points_lco2_enduser_2], how='outer')

In [48]:
pareto_lcoe_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_revenue:Q')
)
pareto_lco2_abatement= alt.Chart(pareto_front_lco2_abatement).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_abatement:Q')
)

pareto_lcoe_investor= alt.Chart(pareto_front_lcoe_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_investor:Q')
)
pareto_lco2_investor= alt.Chart(pareto_front_lco2_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_investor:Q')
)

pareto_lcoe_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_enduser:Q')
)
pareto_lco2_enduser= alt.Chart(pareto_front_lco2_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_enduser:Q')
)

In [49]:
alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env,
        pareto_lcoe_revenue
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env,
        pareto_lco2_abatement
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [50]:
alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env,
        pareto_lcoe_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env,
        pareto_lco2_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [51]:
alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg,
        pareto_lcoe_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg,
        pareto_lco2_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [52]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_revenue,
        right_up_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_revenue,
        right_down_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [53]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [54]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [55]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='plasma',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='plasma',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='turbo',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='turbo',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [56]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='plasma',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='plasma',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='turbo',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='turbo',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [57]:
pareto_front_lcoe_enduser['ress']

0    36.216028
1    54.979190
2    53.388617
3    68.904710
4    77.686106
Name: ress, dtype: float64

### Specifying separate Paretto fronts for economic and environmental dispatch

In [58]:
results_econ= pd.read_csv(
    'Results/Batch5/CaseB/results_econ.csv',
    sep= ';',
)
results_env= pd.read_csv(
    'Results/Batch5/CaseB/results_env.csv',
    sep= ';',
)
results= pd.read_csv(
    'Results/Batch5/CaseB/results.csv',
    sep= ';',
)

In [59]:
# Assuming you want to minimize both objectives or mix
# One front is for min-min and one for min-max, so we end up with left and right part of the Paretto front (this is datapoint dependant)
pareto_points_lcoe_econ_revenue_1 = paretoset.paretoset(results_econ[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_revenue_2 = paretoset.paretoset(results_econ[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_abatement_1 = paretoset.paretoset(results_econ[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_abatement_2 = paretoset.paretoset(results_econ[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_econ_investor_1 = paretoset.paretoset(results_econ[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_investor_2 = paretoset.paretoset(results_econ[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_investor_1 = paretoset.paretoset(results_econ[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_investor_2 = paretoset.paretoset(results_econ[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_econ_enduser_1 = paretoset.paretoset(results_econ[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_enduser_2 = paretoset.paretoset(results_econ[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_enduser_1 = paretoset.paretoset(results_econ[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_enduser_2 = paretoset.paretoset(results_econ[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_econ_revenue = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_revenue_1], results_econ.iloc[pareto_points_lcoe_econ_revenue_2], how='outer')
pareto_front_lco2_econ_abatement = pd.merge(results_econ.iloc[pareto_points_lco2_econ_abatement_1], results_econ.iloc[pareto_points_lco2_econ_abatement_2], how='outer')

pareto_front_lcoe_econ_investor = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_investor_1], results_econ.iloc[pareto_points_lcoe_econ_investor_2], how='outer')
pareto_front_lco2_econ_investor = pd.merge(results_econ.iloc[pareto_points_lco2_econ_investor_1], results_econ.iloc[pareto_points_lco2_econ_investor_2], how='outer')

pareto_front_lcoe_econ_enduser = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_enduser_1], results_econ.iloc[pareto_points_lcoe_econ_enduser_2], how='outer')
pareto_front_lco2_econ_enduser = pd.merge(results_econ.iloc[pareto_points_lco2_econ_enduser_1], results_econ.iloc[pareto_points_lco2_econ_enduser_2], how='outer')



# Assuming you want to minimize both objectives
pareto_points_lcoe_env_revenue_1 = paretoset.paretoset(results_env[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_revenue_2 = paretoset.paretoset(results_env[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_abatement_1 = paretoset.paretoset(results_env[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_abatement_2 = paretoset.paretoset(results_env[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_env_investor_1 = paretoset.paretoset(results_env[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_investor_2 = paretoset.paretoset(results_env[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_investor_1 = paretoset.paretoset(results_env[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_investor_2 = paretoset.paretoset(results_env[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_env_enduser_1 = paretoset.paretoset(results_env[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_enduser_2 = paretoset.paretoset(results_env[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_enduser_1 = paretoset.paretoset(results_env[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_enduser_2 = paretoset.paretoset(results_env[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_env_revenue = pd.merge(results_env.iloc[pareto_points_lcoe_env_revenue_1], results_env.iloc[pareto_points_lcoe_env_revenue_2], how='outer')
pareto_front_lco2_env_abatement = pd.merge(results_env.iloc[pareto_points_lco2_env_abatement_1], results_env.iloc[pareto_points_lco2_env_abatement_2], how='outer')

pareto_front_lcoe_env_investor = pd.merge(results_env.iloc[pareto_points_lcoe_env_investor_1], results_env.iloc[pareto_points_lcoe_env_investor_2], how='outer')
pareto_front_lco2_env_investor = pd.merge(results_env.iloc[pareto_points_lco2_env_investor_1], results_env.iloc[pareto_points_lco2_env_investor_2], how='outer')

pareto_front_lcoe_env_enduser = pd.merge(results_env.iloc[pareto_points_lcoe_env_enduser_1], results_env.iloc[pareto_points_lcoe_env_enduser_2], how='outer')
pareto_front_lco2_env_enduser = pd.merge(results_env.iloc[pareto_points_lco2_env_enduser_1], results_env.iloc[pareto_points_lco2_env_enduser_2], how='outer')

In [60]:
lcoe_econ_investor= alt.Chart(pareto_front_lcoe_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_env_investor= alt.Chart(pareto_front_lcoe_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_econ_enduser= alt.Chart(pareto_front_lcoe_econ_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)
lcoe_env_enduser= alt.Chart(pareto_front_lcoe_env_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)


lco2_econ_investor= alt.Chart(pareto_front_lco2_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_env_investor= alt.Chart(pareto_front_lco2_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_econ_enduser= alt.Chart(pareto_front_lco2_econ_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)
lco2_env_enduser= alt.Chart(pareto_front_lco2_env_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)


alt.hconcat(
    alt.layer(
        lcoe_econ_investor,
        lcoe_econ_enduser,
        lcoe_env_investor,
        lcoe_env_enduser
    ),
    alt.layer(
        lco2_econ_investor,
        lco2_econ_enduser,
        lco2_env_investor,
        lco2_env_enduser
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

## Joining Case A and Case B fronts to compare

In [61]:
break
# apartir de aqui se hace correlacion con el caso A

SyntaxError: 'break' outside loop (1157205122.py, line 1)

In [ ]:
# results_casea_econ= pd.read_csv(
#     'Results/Batch5/CaseA/results_econ.csv',
#     sep= ';'
# )
# results_casea_env= pd.read_csv(
#     'Results/Batch5/CaseA/results_env.csv',
#     sep= ';',
# )
# results_casea= pd.read_csv(
#     'Results/Batch5/CaseA/results.csv',
#     sep= ';',
# )


results_caseb_econ= pd.read_csv(
    'Results/Batch5/CaseB/results_econ.csv',
    sep= ';',
)
results_caseb_env= pd.read_csv(
    'Results/Batch5/CaseB/results_env.csv',
    sep= ';',
)
results_caseb= pd.read_csv(
    'Results/Batch5/CaseB/results.csv',
    sep= ';',
)

In [ ]:
results_caseb_econ['ress'].loc[
    (results_caseb_econ['ress'] <= 20)
    & (results_caseb_econ['ress'] >= 18)
]

In [ ]:
# # Extracting for Case A
# # Assuming you want to minimize both objectives or mix
# # One front is for min-min and one for min-max, so we end up with left and right part of the Paretto front (this is datapoint dependant)
# pareto_points_casea_lcoe_econ_investor_1 = paretoset.paretoset(results_casea_econ[['lcoe_investor', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lcoe_econ_investor_2 = paretoset.paretoset(results_casea_econ[['lcoe_investor', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lco2_econ_investor_1 = paretoset.paretoset(results_casea_econ[['lco2_investor', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lco2_econ_investor_2 = paretoset.paretoset(results_casea_econ[['lco2_investor', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lcoe_econ_enduser_1 = paretoset.paretoset(results_casea_econ[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lcoe_econ_enduser_2 = paretoset.paretoset(results_casea_econ[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lco2_econ_enduser_1 = paretoset.paretoset(results_casea_econ[['lco2_enduser', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lco2_econ_enduser_2 = paretoset.paretoset(results_casea_econ[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# # Extract Pareto-optimal points
# pareto_front_casea_lcoe_econ_investor = pd.merge(results_casea_econ.iloc[pareto_points_casea_lcoe_econ_investor_1], results_casea_econ.iloc[pareto_points_casea_lcoe_econ_investor_2], how='outer')
# pareto_front_casea_lco2_econ_investor = pd.merge(results_casea_econ.iloc[pareto_points_casea_lco2_econ_investor_1], results_casea_econ.iloc[pareto_points_casea_lco2_econ_investor_2], how='outer')
# pareto_front_casea_lcoe_econ_enduser = pd.merge(results_casea_econ.iloc[pareto_points_casea_lcoe_econ_enduser_1], results_casea_econ.iloc[pareto_points_casea_lcoe_econ_enduser_2], how='outer')
# pareto_front_casea_lco2_econ_enduser = pd.merge(results_casea_econ.iloc[pareto_points_casea_lco2_econ_enduser_1], results_casea_econ.iloc[pareto_points_casea_lco2_econ_enduser_2], how='outer')



# # Assuming you want to minimize both objectives
# pareto_points_casea_lcoe_env_investor_1 = paretoset.paretoset(results_casea_env[['lcoe_investor', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lcoe_env_investor_2 = paretoset.paretoset(results_casea_env[['lcoe_investor', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lco2_env_investor_1 = paretoset.paretoset(results_casea_env[['lco2_investor', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lco2_env_investor_2 = paretoset.paretoset(results_casea_env[['lco2_investor', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lcoe_env_enduser_1 = paretoset.paretoset(results_casea_env[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lcoe_env_enduser_2 = paretoset.paretoset(results_casea_env[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
# pareto_points_casea_lco2_env_enduser_1 = paretoset.paretoset(results_casea_env[['lco2_enduser', 'ress']].values, sense=["min", "min"])
# pareto_points_casea_lco2_env_enduser_2 = paretoset.paretoset(results_casea_env[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# # Extract Pareto-optimal points
# pareto_front_casea_lcoe_env_investor = pd.merge(results_casea_env.iloc[pareto_points_casea_lcoe_env_investor_1], results_casea_env.iloc[pareto_points_casea_lcoe_env_investor_2], how='outer')
# pareto_front_casea_lco2_env_investor = pd.merge(results_casea_env.iloc[pareto_points_casea_lco2_env_investor_1], results_casea_env.iloc[pareto_points_casea_lco2_env_investor_2], how='outer')
# pareto_front_casea_lcoe_env_enduser = pd.merge(results_casea_env.iloc[pareto_points_casea_lcoe_env_enduser_1], results_casea_env.iloc[pareto_points_casea_lcoe_env_enduser_2], how='outer')
# pareto_front_casea_lco2_env_enduser = pd.merge(results_casea_env.iloc[pareto_points_casea_lco2_env_enduser_1], results_casea_env.iloc[pareto_points_casea_lco2_env_enduser_2], how='outer')





# Extracting for Case B
# Assuming you want to minimize both objectives or mix
# One front is for min-min and one for min-max, so we end up with left and right part of the Paretto front (this is datapoint dependant)
pareto_points_caseb_lcoe_econ_investor_1 = paretoset.paretoset(results_caseb_econ[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lcoe_econ_investor_2 = paretoset.paretoset(results_caseb_econ[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lco2_econ_investor_1 = paretoset.paretoset(results_caseb_econ[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lco2_econ_investor_2 = paretoset.paretoset(results_caseb_econ[['lco2_investor', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lcoe_econ_enduser_1 = paretoset.paretoset(results_caseb_econ[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lcoe_econ_enduser_2 = paretoset.paretoset(results_caseb_econ[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lco2_econ_enduser_1 = paretoset.paretoset(results_caseb_econ[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lco2_econ_enduser_2 = paretoset.paretoset(results_caseb_econ[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_caseb_lcoe_econ_investor = pd.merge(results_caseb_econ.iloc[pareto_points_caseb_lcoe_econ_investor_1], results_caseb_econ.iloc[pareto_points_caseb_lcoe_econ_investor_2], how='outer')
pareto_front_caseb_lco2_econ_investor = pd.merge(results_caseb_econ.iloc[pareto_points_caseb_lco2_econ_investor_1], results_caseb_econ.iloc[pareto_points_caseb_lco2_econ_investor_2], how='outer')
pareto_front_caseb_lcoe_econ_enduser = pd.merge(results_caseb_econ.iloc[pareto_points_caseb_lcoe_econ_enduser_1], results_caseb_econ.iloc[pareto_points_caseb_lcoe_econ_enduser_2], how='outer')
pareto_front_caseb_lco2_econ_enduser = pd.merge(results_caseb_econ.iloc[pareto_points_caseb_lco2_econ_enduser_1], results_caseb_econ.iloc[pareto_points_caseb_lco2_econ_enduser_2], how='outer')



# Assuming you want to minimize both objectives
pareto_points_caseb_lcoe_env_investor_1 = paretoset.paretoset(results_caseb_env[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lcoe_env_investor_2 = paretoset.paretoset(results_caseb_env[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lco2_env_investor_1 = paretoset.paretoset(results_caseb_env[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lco2_env_investor_2 = paretoset.paretoset(results_caseb_env[['lco2_investor', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lcoe_env_enduser_1 = paretoset.paretoset(results_caseb_env[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lcoe_env_enduser_2 = paretoset.paretoset(results_caseb_env[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_caseb_lco2_env_enduser_1 = paretoset.paretoset(results_caseb_env[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_caseb_lco2_env_enduser_2 = paretoset.paretoset(results_caseb_env[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_caseb_lcoe_env_investor = pd.merge(results_caseb_env.iloc[pareto_points_caseb_lcoe_env_investor_1], results_caseb_env.iloc[pareto_points_caseb_lcoe_env_investor_2], how='outer')
pareto_front_caseb_lco2_env_investor = pd.merge(results_caseb_env.iloc[pareto_points_caseb_lco2_env_investor_1], results_caseb_env.iloc[pareto_points_caseb_lco2_env_investor_2], how='outer')
pareto_front_caseb_lcoe_env_enduser = pd.merge(results_caseb_env.iloc[pareto_points_caseb_lcoe_env_enduser_1], results_caseb_env.iloc[pareto_points_caseb_lcoe_env_enduser_2], how='outer')
pareto_front_caseb_lco2_env_enduser = pd.merge(results_caseb_env.iloc[pareto_points_caseb_lco2_env_enduser_1], results_caseb_env.iloc[pareto_points_caseb_lco2_env_enduser_2], how='outer')

In [ ]:
# pareto_front_casea_lcoe_econ_investor
# pareto_front_casea_lco2_econ_investor
# pareto_front_casea_lcoe_econ_enduser
# pareto_front_casea_lco2_econ_enduser
# pareto_front_casea_lcoe_env_investor
# pareto_front_casea_lco2_env_investor
# pareto_front_casea_lcoe_env_enduser
# pareto_front_casea_lco2_env_enduser

# pareto_front_caseb_lcoe_econ_investor
# pareto_front_caseb_lco2_econ_investor
# pareto_front_caseb_lcoe_econ_enduser
# pareto_front_caseb_lco2_econ_enduser
# pareto_front_caseb_lcoe_env_investor
# pareto_front_caseb_lco2_env_investor
# pareto_front_caseb_lcoe_env_enduser
# pareto_front_caseb_lco2_env_enduser









# lcoe_casea_econ_investor= alt.Chart(pareto_front_casea_lcoe_econ_investor).mark_line(
#     strokeDash= (5, 5),
#     color= 'red',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lcoe_investor:Q'
# )
# lcoe_casea_econ_enduser= alt.Chart(pareto_front_casea_lcoe_econ_enduser).mark_line(
#     strokeDash= (5, 5),
#     color= 'red',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lcoe_enduser:Q'
# )
# lcoe_casea_env_investor= alt.Chart(pareto_front_casea_lcoe_env_investor).mark_line(
#     strokeDash= (5, 5),
#     color= 'green',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lcoe_investor:Q'
# )
# lcoe_casea_env_enduser= alt.Chart(pareto_front_casea_lcoe_env_enduser).mark_line(
#     strokeDash= (5, 5),
#     color= 'green',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lcoe_enduser:Q'
# )
lcoe_caseb_econ_investor= alt.Chart(pareto_front_caseb_lcoe_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_caseb_econ_enduser= alt.Chart(pareto_front_caseb_lcoe_econ_enduser).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)
lcoe_caseb_env_investor= alt.Chart(pareto_front_caseb_lcoe_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_caseb_env_enduser= alt.Chart(pareto_front_caseb_lcoe_env_enduser).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)



# lco2_casea_econ_investor= alt.Chart(pareto_front_casea_lco2_econ_investor).mark_line(
#     strokeDash= (5, 5),
#     color= 'red',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lco2_investor:Q'
# )
# lco2_casea_econ_enduser= alt.Chart(pareto_front_casea_lco2_econ_enduser).mark_line(
#     strokeDash= (5, 5),
#     color= 'red',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lco2_enduser:Q'
# )
# lco2_casea_env_investor= alt.Chart(pareto_front_casea_lco2_env_investor).mark_line(
#     strokeDash= (5, 5),
#     color= 'green',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lco2_investor:Q'
# )
# lco2_casea_env_enduser= alt.Chart(pareto_front_casea_lco2_env_enduser).mark_line(
#     strokeDash= (5, 5),
#     color= 'green',
#     opacity= 0.75
# ).encode(
#     x= 'ress:Q',
#     y= 'lco2_enduser:Q'
# )
lco2_caseb_econ_investor= alt.Chart(pareto_front_caseb_lco2_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_caseb_econ_enduser= alt.Chart(pareto_front_caseb_lco2_econ_enduser).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)
lco2_caseb_env_investor= alt.Chart(pareto_front_caseb_lco2_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_caseb_env_enduser= alt.Chart(pareto_front_caseb_lco2_env_enduser).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)








alt.vconcat(
    alt.hconcat(
        alt.layer(
            lcoe_casea_econ_investor,
            lcoe_casea_env_investor,
            lcoe_caseb_econ_investor,
            lcoe_caseb_env_investor
            ),
        alt.layer(
            lco2_casea_econ_investor,
            lco2_casea_env_investor,
            lco2_caseb_econ_investor,
            lco2_caseb_env_investor
            )
    ),
    alt.hconcat(
        alt.layer(
            lcoe_casea_econ_enduser,
            lcoe_casea_env_enduser,
            lcoe_caseb_econ_enduser,
            lcoe_caseb_env_enduser
        ),
        alt.layer(
            lco2_casea_econ_enduser,
            lco2_casea_env_enduser,
            lco2_caseb_econ_enduser,
            lco2_caseb_env_enduser
        )
    )
)